In [7]:


from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [8]:
!pip install opencv-python

In [9]:
import os
import cv2
from sklearn.utils import shuffle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.utils import to_categorical

def get_images(directory):
    Images = []
    Labels = []  # 0 non_diabetic , 1 for diabetic
    for labels in os.listdir(directory): 
        for image_file in os.listdir(directory+labels): 
            image = cv2.imread(directory+labels+r'/'+image_file) 
            image = cv2.resize(image,(224,224)) 
            Images.append(image)
            Labels.append(labels)
    return shuffle(Images,Labels,random_state=817328462)

ImagesC, LabelsC = get_images("Data\\Conjuctiva\\")
ImagesC = np.array(ImagesC)
LabelsC = np.array(LabelsC)

label_encoder = LabelEncoder()
YC = np.array(label_encoder.fit_transform(LabelsC))

ImagesI, LabelsI = get_images("Data\\Iris - Small\\")
ImagesI = np.array(ImagesI)
LabelsI = np.array(LabelsI)

label_encoder = LabelEncoder()
YI = np.array(label_encoder.fit_transform(LabelsI))

combined_features = np.concatenate((ImagesC, ImagesI), axis=0)
combined_labels = np.concatenate((YC, YI), axis=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(combined_features, combined_labels, test_size=0.15, shuffle=True)

In [11]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# load model without classifier layers
base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in base model
for layer in base_model.layers:
    layer.trainable = False

# add new classifier layers
flat1 = Flatten()(base_model.output)
class1 = Dense(1024, activation='relu')(flat1)
dropout1 = Dropout(0.5)(class1)
class2 = Dense(1024, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(class2)
output = Dense(2, activation='softmax')(dropout2)

# define new model
model = Model(inputs=base_model.inputs, outputs=output)

# compile model
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Convert labels to categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# Early stopping and model checkpointing
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# fit model
history = model.fit(datagen.flow(X_train, y_train_cat, batch_size=32), 
                    validation_data=(X_test, y_test_cat), 
                    epochs=100, verbose=0, callbacks=[es, mc])

# load the saved model
from tensorflow.keras.models import load_model
saved_model = load_model('best_model.h5')

# predict
y_pred = saved_model.predict(X_test)

# convert probabilities to class labels
y_pred = np.argmax(y_pred, axis=1)

# evaluate model
_, acc = saved_model.evaluate(X_test, y_test_cat, verbose=0)
print('Test Accuracy: %.3f' % (acc * 100.0))

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix
print(confusion_matrix(y_test, y_pred))

94765736/94765736 [==============================] - 10s 0us/step

Epoch 1: val_accuracy improved from -inf to 0.78488, saving model to best_model.h5

Epoch 2: val_accuracy did not improve from 0.78488

Epoch 3: val_accuracy did not improve from 0.78488

Epoch 4: val_accuracy improved from 0.78488 to 0.80233, saving model to best_model.h5

Epoch 5: val_accuracy improved from 0.80233 to 0.80814, saving model to best_model.h5

Epoch 6: val_accuracy improved from 0.80814 to 0.83721, saving model to best_model.h5

Epoch 7: val_accuracy improved from 0.83721 to 0.84302, saving model to best_model.h5

Epoch 8: val_accuracy did not improve from 0.84302

Epoch 9: val_accuracy did not improve from 0.84302

Epoch 10: val_accuracy did not improve from 0.84302

Epoch 11: val_accuracy improved from 0.84302 to 0.86047, saving model to best_model.h5

Epoch 12: val_accuracy did not improve from 0.86047

Epoch 13: val_accuracy did not improve from 0.86047

Epoch 14: val_accuracy did not improve from 0.